In [75]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from scipy import stats
from IPython.display import display

def export(df: pd.DataFrame, path_no_dot: str, index=True):
    df.to_csv(f'exports/{path_no_dot}.csv', index=index)
    df.to_excel(f'exports/{path_no_dot}.xlsx', index=index)

In [76]:
# data initialization

# needs ppt_analysis.ipynb to be ran first
players_table = pd.read_csv('exports/players_table.csv')
matches_table = pd.read_csv('data/matches.csv').sort_values(['event', 'match_number']).reset_index(drop=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,date,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2
0,12/4/2021,2021_2s,1,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3
1,12/4/2021,2021_2s,2,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3
2,12/4/2021,2021_2s,3,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3
3,12/4/2021,2021_2s,4,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0
4,12/4/2021,2021_2s,5,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6
5,12/4/2021,2021_2s,6,True,True,Ann Hall,Anna Brown,Paul Bartenfeld,Leah Baetcke,6,4
6,12/4/2021,2021_2s,7,False,False,Rose Roché,Aaron Carter,Coby Lovelace,Jason Jackson,5,6
7,12/4/2021,2021_2s,8,False,False,Kevin Cooper,Roman Ramirez,Rohan Chowla,Levin Lee,5,6
8,12/4/2021,2021_2s,9,True,True,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,5
9,12/4/2021,2021_2s,10,True,True,Rose Roché,Aaron Carter,Jack Massingill,Will Simpson,5,6


In [77]:
def expected_score(ratingA, ratingB): # expected difference in cups hit
    # sigmoid vertical bounds
    yrange = 12
    ymin = 6
    # game variables
    expected_cups = 2
    normal_elo_difference = 400

    # score is the expected difference in cups scored,
        # if this function spits out -5, then players of those two ratings should end in a difference of 5 cups
    # this function is a sigmoid function from -6 to 6
    # this function says an elo difference of "normal_elo_difference" should result in absolute difference in "expected_cups"
        # this works: an elo difference of 400 should result in a score difference of 2
    return (yrange / (1 + np.power(expected_cups, (ratingB - ratingA) / normal_elo_difference))) - ymin

def rating_change(score, expected_score):
    # the k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, but with few games, we want to change it frequently
    K = 12
    return K * (score - expected_score)

# ELO INITIALIZATION
starting_elo = 1200.0
elo = dict()
elo_time = np.zeros([len(players_table['player']), len(matches_table) + 1])
d_elo_time = np.zeros([len(players_table['player']), len(matches_table) + 1])
for player in players_table['player']:
    elo.update({player: starting_elo})
    elo_time[players_table['player'][players_table['player'] == player].index[0], 0] = starting_elo

# ALGORITHM
for (i, row) in matches_table.iterrows():
    elos_t1 = list()
    elos_t2 = list()

    elos_t1.append(elo[row.t1p1])
    elos_t2.append(elo[row.t2p1])

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):
        elos_t1.append(elo[row.t1p2])
        elos_t2.append(elo[row.t2p2])

    elo_t1 = np.mean(elos_t1)
    elo_t2 = np.mean(elos_t2)

    win_prob_t1 = expected_score(elo_t1, elo_t2)
    win_prob_t2 = expected_score(elo_t2, elo_t1)

    rating_change_t1p1 = rating_change(row.score1 - row.score2, win_prob_t1)
    rating_change_t2p1 = rating_change(row.score2 - row.score1, win_prob_t2)

    elo[row.t1p1] += rating_change_t1p1
    elo[row.t2p1] += rating_change_t2p1
    
    d_elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = rating_change_t1p1
    d_elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = rating_change_t2p1

    elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = elo[row.t1p1]
    elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = elo[row.t2p1]

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):

        rating_change_t1p2 = rating_change(row.score1 - row.score2, win_prob_t1)
        rating_change_t2p2 = rating_change(row.score2 - row.score1, win_prob_t2)

        elo[row.t1p2] += rating_change_t1p2
        elo[row.t2p2] += rating_change_t2p2

        d_elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = rating_change_t1p2
        d_elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = rating_change_t2p2
        
        elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = elo[row.t1p2]
        elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = elo[row.t2p2]

players_table['current_elo'] = players_table['player'].map(elo)

player_elo_table = players_table[['player', 'current_elo']].sort_values('current_elo', ascending=False).reset_index(drop=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(player_elo_table)

export(player_elo_table, 'player_elo_table', False)

,player,current_elo
0,Levin Lee,12191.624867
1,Aaron Carter,12185.287081
2,Roman Ramirez,12176.249823
3,Rohan Chowla,12077.890315
4,Kevin Cooper,12071.813140
5,Will Simpson,12044.496624
6,Nathan Snow,12034.327352
7,Gabe Silverstein,12033.467836
8,Coby Lovelace,12030.531071
9,Jack Massingill,12028.341893


In [78]:
print("Expected score between 1400 and 1000", expected_score(1400, 1000))
print()
print("A much higher ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1400, 1000)))
print("A much higher ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1400, 1000)))
print("A much higher ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1400, 1000)))
print("A much higher ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1400, 1000)))
print()
print("A closely higher ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1250, 1200)))
print("A closely higher ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1250, 1200)))
print("A closely higher ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1250, 1200)))
print("A closely higher ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1250, 1200)))
print()
print("A closely lower ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1200, 1250)))
print("A closely lower ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1200, 1250)))
print("A closely lower ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1200, 1250)))
print("A closely lower ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1200, 1250)))
print()
print("A much lower ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1000, 1400)))
print("A much lower ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1000, 1400)))
print("A much lower ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1000, 1400)))
print("A much lower ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1000, 1400)))
print()

Expected score between 1400 and 1000 2.0

A much higher ELO, B barely wins: A's change is -36.0
A much higher ELO, A barely wins: A's change is -12.0
A much higher ELO, B hugely wins: A's change is -84.0
A much higher ELO, A hugely wins: A's change is 36.0

A closely higher ELO, B barely wins: A's change is -15.117212459956022
A closely higher ELO, A barely wins: A's change is 8.882787540043978
A closely higher ELO, B hugely wins: A's change is -63.11721245995602
A closely higher ELO, A hugely wins: A's change is 56.88278754004398

A closely lower ELO, B barely wins: A's change is -8.882787540043967
A closely lower ELO, A barely wins: A's change is 15.117212459956033
A closely lower ELO, B hugely wins: A's change is -56.882787540043964
A closely lower ELO, A hugely wins: A's change is 63.117212459956036

A much lower ELO, B barely wins: A's change is 12.0
A much lower ELO, A barely wins: A's change is 36.0
A much lower ELO, B hugely wins: A's change is -36.0
A much lower ELO, A hugely 

In [79]:
elo_time_table = pd.concat([players_table.player, pd.DataFrame(elo_time)], axis=1).T
elo_time_table.columns = players_table.player
elo_time_table = elo_time_table.iloc[2:,:]
elo_time_table.replace(0.0, np.nan, inplace=True)

d_elo_time_table = pd.concat([players_table.player, pd.DataFrame(d_elo_time)], axis=1).T
d_elo_time_table.columns = players_table.player
d_elo_time_table = d_elo_time_table.iloc[2:,:]
d_elo_time_table.replace(0.0, np.nan, inplace=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(elo_time_table)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(d_elo_time_table)

export(elo_time_table, 'elo_time_table', True)

player,Aaron Carter,Ann Hall,Anna Brown,Brian Tafazoli,Carla Betancourt,Cason Duszak,Cassie Deering,Coby Lovelace,Evan Sooklal,Gabe Silverstein,Helen Dunn,Jack Massingill,Jason Jackson,Kevin Cooper,Kristian Banlaoi,Leah Baetcke,Levin Lee,Matthew Rusten,Nathan Snow,Noah Dale,Paul Bartenfeld,Piper Parker,Reagan Fryatt,Rohan Chowla,Roman Ramirez,Rose Roché,Sam Carswell-Tellis,Will Simpson,Yvonne Nguyen
1,12036.000000,NaN,NaN,11964.0,NaN,NaN,NaN,NaN,11964.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12036.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12036.000000,NaN,NaN,NaN,11964.000000,12036.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11964.000000,NaN
3,NaN,11964.000000,11964.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12036.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12036.000000,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11928.000000,12072.000000,NaN,NaN,NaN,11928.000000,NaN,NaN,12072.000000,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,11940.0,NaN,NaN,NaN,NaN,11940.000000,NaN,NaN,11988.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11988.000000,NaN
6,NaN,11985.754931,11985.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11906.245069,NaN,NaN,NaN,NaN,11906.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,12024.000000,NaN,NaN,NaN,NaN,NaN,NaN,12048.000000,NaN,NaN,NaN,NaN,12048.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12024.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12026.245069,NaN,NaN,12081.754931,NaN,NaN,NaN,NaN,NaN,NaN,12081.754931,12026.245069,NaN,NaN,NaN,NaN
9,NaN,11976.279802,11976.279802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12035.720198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12035.720198,NaN,NaN,NaN,NaN
10,12009.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12002.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12009.754931,NaN,12002.245069,NaN


player,Aaron Carter,Ann Hall,Anna Brown,Brian Tafazoli,Carla Betancourt,Cason Duszak,Cassie Deering,Coby Lovelace,Evan Sooklal,Gabe Silverstein,Helen Dunn,Jack Massingill,Jason Jackson,Kevin Cooper,Kristian Banlaoi,Leah Baetcke,Levin Lee,Matthew Rusten,Nathan Snow,Noah Dale,Paul Bartenfeld,Piper Parker,Reagan Fryatt,Rohan Chowla,Roman Ramirez,Rose Roché,Sam Carswell-Tellis,Will Simpson,Yvonne Nguyen
1,36.000000,NaN,NaN,-36.0,NaN,NaN,NaN,NaN,-36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN,-36.000000,36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-36.000000,NaN
3,NaN,-36.000000,-36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-72.000000,72.000000,NaN,NaN,NaN,-72.000000,NaN,NaN,72.000000,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,-24.0,NaN,NaN,NaN,NaN,-24.000000,NaN,NaN,24.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.000000,NaN
6,NaN,21.754931,21.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-21.754931,NaN,NaN,NaN,NaN,-21.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-12.000000,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.754931,NaN,NaN,9.754931,NaN,NaN,NaN,NaN,NaN,NaN,9.754931,-9.754931,NaN,NaN,NaN,NaN
9,NaN,-9.475130,-9.475130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.475130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.475130,NaN,NaN,NaN,NaN
10,-14.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.245069,NaN,14.245069,NaN


In [80]:
import plotly.graph_objects as go

fig = go.Figure()

ranking = 1
for (player,current_rating) in sorted(elo.items(), key=lambda x:x[1], reverse=True):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        line_shape='hv',
        connectgaps=True,
        text=d_elo_time_table[player],
    ))
    ranking += 1

# generalized
vrect_colors = ['green', 'red', 'yellow', 'blue']
seasons = ['2021_2s', '2022_1s', '2022_2s', '2023_1s']
season_labels = ['2021<br>Doubles', '2022<br>Singles', '2022<br>Doubles', '2023<br>Singles']

for (i, season) in enumerate(seasons):
    fig.add_vrect(
        annotation_text=season_labels[i],
        annotation_position="top left",
        x0=matches_table['event'][matches_table['event'] == season].index[0] + 0.5,
        x1=matches_table['event'][matches_table['event'] == season].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0,
    )


fig.update_layout(
    title='<b>UVA Drumline Pong ELO Rating System</b> <br><i>by Roman Ramirez<i>',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>',
)

team1_list = list()
team2_list = list()
for (i, row) in matches_table.iterrows():
    if type(row.t1p2) is not str:
        team1_list.append(row.t1p1)
    else:
        team1_list.append(row.t1p1 + " and " + row.t1p2)

    if type(row.t2p2) is not str:
        team2_list.append(row.t2p1)
    else:
        team2_list.append(row.t2p1 + " and " + row.t2p2)

customdata = np.stack((team1_list, team2_list, matches_table['score1'], matches_table['score2'], matches_table['date']), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '<b>ELO Change:</b> %{text:,.0f}<br>' + 
    '<b>New ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate
)

fig.show()
fig.write_html("index.html")

In [81]:

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,date,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2
0,12/4/2021,2021_2s,1,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3
1,12/4/2021,2021_2s,2,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3
2,12/4/2021,2021_2s,3,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3
3,12/4/2021,2021_2s,4,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0
4,12/4/2021,2021_2s,5,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6
5,12/4/2021,2021_2s,6,True,True,Ann Hall,Anna Brown,Paul Bartenfeld,Leah Baetcke,6,4
6,12/4/2021,2021_2s,7,False,False,Rose Roché,Aaron Carter,Coby Lovelace,Jason Jackson,5,6
7,12/4/2021,2021_2s,8,False,False,Kevin Cooper,Roman Ramirez,Rohan Chowla,Levin Lee,5,6
8,12/4/2021,2021_2s,9,True,True,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,5
9,12/4/2021,2021_2s,10,True,True,Rose Roché,Aaron Carter,Jack Massingill,Will Simpson,5,6
